# update2: add new sources

just add them all. type is `journal` unless `conference` appears in title.lower()

remember to add issnl to issns and move to front

In [1]:
import sys

In [2]:
sys.path.append('../..')

In [3]:
from datetime import datetime, timezone
import json
from tqdm import tqdm
from pathlib import Path
import gzip

In [4]:
import pandas as pd
import numpy as np

In [5]:
from app import db
from models import Source
from sqlalchemy import text
from sqlalchemy.orm import Load
from sqlalchemy.exc import MultipleResultsFound

139886361092480: loading merged_into_institutions_dict
139886361092480: loading valid concept IDs
139886361092480: loading valid topic IDs


In [6]:
%%time
sq = """select * from issn_audit_20240321.issn_ic_datafile_202402"""
df_datafile = pd.read_sql_query(sq, db.engine)

CPU times: user 599 ms, sys: 127 ms, total: 726 ms
Wall time: 1.98 s


In [7]:
# get dict of issnl to issn
x = df_datafile.dropna(subset=['issnl'])
x = x[x['category'].isin(['Register', 'Work', 'Free', 'Validation Request'])]
x = x[['issn', 'issnl']]
issnl_to_issn = {}
ignore = [
    '0000-0000',
]
x = x[~(x['issnl'].isin(ignore))]
for name, gbdf in x.groupby('issnl'):
    if name in ignore:
        continue
    issnl_to_issn[name] = gbdf['issn'].tolist()
len(issnl_to_issn)

140984

In [8]:
sq = """select issnl from issn_audit_20240321.update1_issnl_notfound"""
df_notfound = pd.read_sql_query(sq, db.engine)
subset = df_datafile.merge(df_notfound, how='inner', on='issnl')
len(subset)

9666

In [9]:
subset = subset[subset['resolved']==False]
len(subset)

3047

In [10]:
subset['issnl'].duplicated().sum()

721

In [11]:
subset = subset[subset['category'].isin(['Register', 'Work', 'Free', 'Validation Request'])]
len(subset)

3047

In [12]:
def add_new_source(issnl, issn_list, title, source_type='journal', updated_date=None, session=None):
    if session is None:
        session = db.session
    if updated_date is None:
        updated_date = datetime.now(timezone.utc).isoformat()

    # make updates
    new_issns = json.dumps(issn_list)
    new_display_name = title
    new_type = source_type
    source = Source(
        display_name = new_display_name,
        issn = issnl,
        issns = new_issns,
        issns_text_array = issn_list,
        type = new_type,
        created_date = updated_date,
        updated_date = updated_date,
    )
    session.add(source)
    session.commit()
    session.refresh(source)
    source_id = int(source.id)

    # insert into log table
    sq = """INSERT INTO issn_audit_20240321.update2
        (issnl, new_issns, new_display_name, updated_date, source_id, new_type)
        VALUES(:issnl, :new_issns, :new_display_name, :updated_date, :source_id, :new_type);
        """
    session.execute(text(sq), {
        'issnl': issnl,
        'new_issns': new_issns,
        'new_display_name': new_display_name,
        'updated_date': updated_date,
        'source_id': source_id,
        'new_type': new_type,
    }) 
    return source_id

In [26]:
# update db
try:
    for issnl, gbdf in tqdm(subset.groupby('issnl'), total=subset['issnl'].nunique()):
        updated_date = datetime.now(timezone.utc).isoformat()
        issn_list = issnl_to_issn[issnl]
        if issnl not in issn_list:
            issn_list.append(issnl)
        # move issnl to the front
        issn_list.insert(0, issn_list.pop(issn_list.index(issnl)))
        row = gbdf[gbdf['issnl']==issn_list[0]].iloc[0]
            
        title = row['title']
        if not title:
            continue
        if 'conference' in title.lower():
            source_type = 'conference'
        elif 'book series' in title.lower():
            source_type = 'book series'
        else:
            source_type = 'journal'
        source_id = add_new_source(issnl, issn_list, title=title, source_type=source_type, updated_date=updated_date, session=db.session)

        for issn in issn_list:
            sq = """update issn_audit_20240321.issn_ic_datafile_202402 set resolved = true
                where issn = :issn
                """
            db.session.execute(text(sq), {
                'issn': issn,
            })

        # # commit for each new source
        db.session.commit()
finally:
    db.session.close()

 74%|███████▍  | 6542/8868 [50:59<18:07,  2.14it/s]  


AttributeError: 'NoneType' object has no attribute 'lower'

In [13]:
# update db
try:
    for issnl, gbdf in tqdm(subset.groupby('issnl'), total=subset['issnl'].nunique()):
        updated_date = datetime.now(timezone.utc).isoformat()
        issn_list = issnl_to_issn[issnl]
        if issnl not in issn_list:
            issn_list.append(issnl)
        # move issnl to the front
        issn_list.insert(0, issn_list.pop(issn_list.index(issnl)))
        row = gbdf[gbdf['issnl']==issn_list[0]].iloc[0]
            
        title = row['title']
        if not title:
            continue
        if 'conference' in title.lower():
            source_type = 'conference'
        elif 'book series' in title.lower():
            source_type = 'book series'
        else:
            source_type = 'journal'
        source_id = add_new_source(issnl, issn_list, title=title, source_type=source_type, updated_date=updated_date, session=db.session)

        for issn in issn_list:
            sq = """update issn_audit_20240321.issn_ic_datafile_202402 set resolved = true
                where issn = :issn
                """
            db.session.execute(text(sq), {
                'issn': issn,
            })

        # # commit for each new source
        db.session.commit()
finally:
    db.session.close()

100%|██████████| 2326/2326 [20:13<00:00,  1.92it/s]
